<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/pi05-test/nine_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nine Test

This test is about understanding how the batch for training are being created. And I don't want to make the policy, I mean they should be independent until a certain extain.

In [2]:
from huggingface_hub import login
login()

In [3]:
!

Cloning into 'XHUMAN'...
remote: Enumerating objects: 2060, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 2060 (delta 166), reused 171 (delta 95), pack-reused 1794 (from 2)
Receiving objects: 100% (2060/2060), 7.81 MiB | 5.95 MiB/s, done.
Resolving deltas: 100% (1309/1309), done.


In [4]:
%cd XHUMAN

/content/XHUMAN


In [5]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 30.52s
Prepared 46 packages in 22.36s
Uninstalled 15 packages in 980ms
Installed 46 packages in 214ms
 + async-lru==2.0.5
 + av==15.1.0
 + comm==0.2.3
 + deepdiff==8.6.1
 - diffusers==0.36.0
 + diffusers==0.35.2
 + draccus==0.10.0
 + evdev==1.9.2
 + faker==40.1.0
 + feetech-servo-sdk==1.0.0
 - huggingface-hub==0.36.0
 + huggingface-hub==0.35.3
 + inquirerpy==0.3.4
 - ipykernel==6.17.1
 + ipykernel==7.1.0
 + jedi==0.19.2
 + json5==0.13.0
 + jsonlines==4.0.0
 + jupyter==1.1.1
 - jupyter-client==7.4.9
 + jupyter-client==8.8.0
 + jupyter-lsp==2.3.0
 + jupyterlab==4.5.2
 + jupyterlab-server==2.28.0
 + lerobot==0.4.3 (from git+https://github.com/huggingface/lerobot.git@15724826dd5b18c18656151a02a24ce5fd690c46)
 + mergedeep==1.3.4
 + mypy-extensions==1.1.0
 - notebook==6.5.7
 + notebook==7.5.2
 - nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cusparselt-cu12==0.7.1
 + nvidia-cusparselt-cu12==0.6.3
 - nvi

In [7]:
import torch
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id="NONHUMAN-RESEARCH/TEST_RECORD_ANNOTATIONS",
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

I guess that the configuration defines the action horizon, which is all.

In [11]:
from xhuman.policies.pi05.configuration_pi05 import PI05Config

policy_config = PI05Config(
    device=device,
    repo_id="none/none"
)

In [12]:
from xhuman.configs.train import TrainPipelineConfigXHUMAN

cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config,
    batch_size=50,
    num_workers=4,
    split_ratio=0.8,
)

cfg.validate()

In [13]:
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs

ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
accelerator = Accelerator(step_scheduler_with_optimizer=False, kwargs_handlers=[ddp_kwargs])

In [ ]:
from xhuman.datasets.factory import make_dataset_xhuman
from xhuman.datasets.utils import split_train_eval_episodes

dataset = make_dataset_xhuman(cfg)
episodes = list(range(dataset.meta.total_episodes))
train_episodes, _ = split_train_eval_episodes(episodes, split_ratio=cfg.split_ratio, seed=42)
if 30 in train_episodes:
    train_episodes.remove(30)

In [23]:
cfg.policy

PI05Config(n_obs_steps=1, input_features={'observation.state': PolicyFeature(type=<FeatureType.STATE: 'STATE'>, shape=(14,)), 'observation.images.left': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 376, 672)), 'observation.images.top': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 376, 672)), 'observation.images.right': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 376, 672))}, output_features={'action': PolicyFeature(type=<FeatureType.ACTION: 'ACTION'>, shape=(14,))}, device=device(type='cuda'), use_amp=False, use_peft=False, push_to_hub=True, repo_id='none/none', private=None, tags=None, license=None, pretrained_path=None, alpha=10.0, max_decoding_steps=100, temperature=0.1, eos_token_id=1, paligemma_variant='gemma_2b', action_expert_variant='gemma_300m', dtype='float32', chunk_size=50, n_action_steps=50, max_state_dim=32, max_action_dim=32, num_inference_steps=10, time_sampling_beta_alpha=1.5, time_sampling_beta_beta=1.0, time_sampling_s

In [ ]:
from xhuman.policies.factory import (make_xhuman_pre_post_processors)

# Prepare processor kwargs (same as in train_val_pi05.py lines 332-360)
processor_kwargs = {}
if not cfg.policy.pretrained_path:
    # Only provide dataset_stats when not resuming from saved processor state
    processor_kwargs["dataset_stats"] = dataset.meta.stats

preprocessor, _ = make_xhuman_pre_post_processors(
    policy_cfg=cfg.policy,
    pretrained_path=None,
    **processor_kwargs,
)

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [28]:
from xhuman.datasets.factory import make_dataset_xhuman

cfg.dataset.episodes = train_episodes
dataset = make_dataset_xhuman(cfg)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

In [29]:
from lerobot.datasets.sampler import EpisodeAwareSampler

# Crear dataloader
if hasattr(cfg.policy, "drop_n_last_frames"):
    sampler = EpisodeAwareSampler(
        dataset.meta.episodes["dataset_from_index"],
        dataset.meta.episodes["dataset_to_index"],
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    sampler = None


In [30]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=cfg.num_workers,
    batch_size=cfg.batch_size,
    shuffle=(sampler is None) and not cfg.dataset.streaming,
    sampler=sampler,
    pin_memory=device.type == "cuda",
    drop_last=False,
    prefetch_factor=2 if cfg.num_workers > 0 else None,
)

In [33]:
from lerobot.datasets.utils import cycle
dataloader = accelerator.prepare(dataloader)
dl_iter = cycle(dataloader)

## Understanding Batch Creation for Policy Forward Pass

This section explains how batches are created and processed before being passed to `policy.forward()` in `train_val_pi05.py`.

### Batch Creation Flow (from train_val_pi05.py)

1. **Raw Batch from DataLoader** (line 484):
   ```python
   batch = next(dl_iter)  # Raw batch from dataset
   ```

2. **Preprocessing** (line 485):
   ```python
   batch = preprocessor(batch)  # Transform batch for policy
   ```

3. **Policy Forward Pass** (line 108):
   ```python
   losses, dict_losses = policy.forward(batch)
   ```

### Preprocessor Pipeline Steps

The preprocessor (created at lines 355-360 in train_val_pi05.py) applies these steps in order:

1. **RenameObservationsProcessorStep**: Renames features to match pretrained configs
2. **AddBatchDimensionProcessorStep**: Adds batch dimension if needed
3. **NormalizerProcessorStep**: Normalizes features using dataset stats
   - Uses `cfg.policy.normalization_mapping` to determine normalization mode
   - STATE and ACTION: QUANTILES normalization (to [-1, 1])
   - VISUAL: IDENTITY (no normalization)
4. **Pi05PrepareStateTokenizerProcessorStep**: 
   - Pads state to `cfg.policy.max_state_dim` (default: 32)
   - Discretizes normalized state to 256 bins
   - Creates prompt: `"Task: {subtask} State: {discretized_state};\\nAction: "`
5. **TokenizerProcessorStep**: Tokenizes prompt using PaliGemma tokenizer
   - Max length: `cfg.policy.tokenizer_max_length` (default: 200)
   - Padding: "max_length", side: "right"
6. **DeviceProcessorStep**: Moves all tensors to `cfg.policy.device`

### Batch Structure After Preprocessing

The batch passed to `policy.forward()` contains:

- **Images**: Dict keys like `"observation.images.left"`, `"observation.images.top"`, `"observation.images.right"`
  - Shape: `[B, C, H, W]` where H=W=224 (from `cfg.policy.image_resolution`)
  - Values: Normalized to [-1, 1] range
  
- **Image Masks**: Created by policy's `_preprocess_images()` method
  - Shape: `[B]` boolean tensors
  
- **Language Tokens**: Key `"observation.language_tokens"`
  - Shape: `[B, seq_len]` where seq_len = tokenizer_max_length
  - Contains tokenized prompt tokens
  
- **Attention Masks**: Key `"observation.language_attention_mask"`
  - Shape: `[B, seq_len]` boolean tensors
  
- **Actions**: Key `"action"`
  - Shape: `[B, n_action_steps, action_dim]` 
  - `n_action_steps` = `cfg.policy.n_action_steps` (default: 50)
  - Normalized to [-1, 1] range

### Key Configuration Parameters

From `cfg.policy` (PI05Config):
- `n_obs_steps`: Number of observation steps (default: 1)
- `n_action_steps`: Action horizon (default: 50)
- `max_state_dim`: Maximum state dimension for padding (default: 32)
- `tokenizer_max_length`: Max sequence length for tokenizer (default: 200)
- `image_resolution`: Image size (default: (224, 224))
- `normalization_mapping`: How to normalize each feature type
- `input_features`: Defines observation features (state, images)
- `output_features`: Defines action features

### DataLoader Configuration

From `cfg` (TrainPipelineConfigXHUMAN):
- `batch_size`: Batch size (default: 50)
- `num_workers`: Number of data loading workers (default: 4)
- `split_ratio`: Train/eval split ratio (default: 0.8)

The DataLoader uses:
- `EpisodeAwareSampler` if `cfg.policy.drop_n_last_frames` is set
- Shuffles episodes but maintains episode boundaries
- `drop_last=False`: Keeps incomplete batches

In [ ]:
# Inspect raw batch from dataloader (before preprocessing)
raw_batch = next(dl_iter)
print("=" * 80)
print("RAW BATCH (from DataLoader, before preprocessor)")
print("=" * 80)
print(f"\nBatch keys: {list(raw_batch.keys())}")
print(f"\nBatch size (number of samples): {len(raw_batch.get('action', raw_batch.get('observation.state', [])))}")

# Show shapes and types of key components
for key, value in raw_batch.items():
    if isinstance(value, torch.Tensor):
        print(f"\n{key}:")
        print(f"  Shape: {value.shape}")
        print(f"  Dtype: {value.dtype}")
        print(f"  Device: {value.device}")
        if value.numel() < 20:  # Only print small tensors
            print(f"  Values: {value}")
    elif isinstance(value, (list, tuple)) and len(value) > 0:
        print(f"\n{key}:")
        print(f"  Type: {type(value)}")
        print(f"  Length: {len(value)}")
        if isinstance(value[0], torch.Tensor):
            print(f"  First element shape: {value[0].shape}")
    else:
        print(f"\n{key}: {type(value)} - {value}")

In [ ]:
# Inspect processed batch (after preprocessor, ready for policy.forward())
processed_batch = preprocessor(raw_batch.copy())
print("=" * 80)
print("PROCESSED BATCH (after preprocessor, ready for policy.forward())")
print("=" * 80)
print(f"\nBatch keys: {list(processed_batch.keys())}")

# Show shapes and types of key components
for key, value in processed_batch.items():
    if isinstance(value, torch.Tensor):
        print(f"\n{key}:")
        print(f"  Shape: {value.shape}")
        print(f"  Dtype: {value.dtype}")
        print(f"  Device: {value.device}")
        if "language" in key.lower() or "token" in key.lower():
            print(f"  Min: {value.min().item()}, Max: {value.max().item()}")
            # Show first few tokens
            if value.dim() >= 2:
                print(f"  First sample tokens (first 20): {value[0, :20]}")
        elif value.numel() < 50:  # Only print small tensors
            print(f"  Values: {value}")
    elif isinstance(value, (list, tuple)) and len(value) > 0:
        print(f"\n{key}:")
        print(f"  Type: {type(value)}")
        print(f"  Length: {len(value)}")
        if isinstance(value[0], torch.Tensor):
            print(f"  First element shape: {value[0].shape}")
    else:
        print(f"\n{key}: {type(value)}")

print("\n" + "=" * 80)
print("KEY BATCH COMPONENTS FOR POLICY.FORWARD():")
print("=" * 80)
print("\n1. Images:")
for key in processed_batch.keys():
    if "image" in key.lower() and isinstance(processed_batch[key], torch.Tensor):
        print(f"   {key}: {processed_batch[key].shape}")

print("\n2. Language Tokens:")
lang_keys = [k for k in processed_batch.keys() if "language" in k.lower() or "token" in k.lower()]
for key in lang_keys:
    if isinstance(processed_batch[key], torch.Tensor):
        print(f"   {key}: {processed_batch[key].shape}")

print("\n3. Actions:")
if "action" in processed_batch:
    print(f"   action: {processed_batch['action'].shape}")

In [ ]:
# Show the exact batch structure that policy.forward() expects
# Based on modeling_pi_05.py lines 2108-2135

print("=" * 80)
print("BATCH STRUCTURE EXPECTED BY policy.forward()")
print("=" * 80)
print("\nFrom modeling_pi_05.py PI05Policy.forward() method:")
print("\nExpected batch keys:")
print("  - Images: observation.images.* (e.g., 'observation.images.left', 'observation.images.top', 'observation.images.right')")
print("  - Language tokens: 'observation.language_tokens'")
print("  - Attention mask: 'observation.language_attention_mask'")
print("  - Actions: 'action'")

print("\n" + "-" * 80)
print("What policy.forward() does with the batch:")
print("-" * 80)
print("1. Calls self._preprocess_images(batch) to extract and process images")
print("   - Returns: images (list of [B, C, H, W] tensors), img_masks (list of [B] bool tensors)")
print("2. Extracts tokens and masks:")
print("   - tokens = batch['observation.language_tokens']  # [B, seq_len]")
print("   - masks = batch['observation.language_attention_mask']  # [B, seq_len]")
print("3. Calls self.prepare_action(batch) to extract and pad actions")
print("   - Returns: actions # [B, n_action_steps, action_dim]")
print("4. Calls self.model.forward(images, img_masks, tokens, masks, actions)")
print("   - Computes ce_loss [B] and flow_matching_loss [B]")
print("5. Returns: ([ce_loss, flow_matching_loss], dict_losses)")

print("\n" + "-" * 80)
print("Configuration used for batch creation:")
print("-" * 80)
print(f"  batch_size: {cfg.batch_size}")
print(f"  n_obs_steps: {cfg.policy.n_obs_steps}")
print(f"  n_action_steps: {cfg.policy.n_action_steps}")
print(f"  max_state_dim: {cfg.policy.max_state_dim}")
print(f"  tokenizer_max_length: {cfg.policy.tokenizer_max_length}")
print(f"  image_resolution: {cfg.policy.image_resolution}")
print(f"  normalization_mapping: {cfg.policy.normalization_mapping}")
print(f"  input_features: {list(cfg.policy.input_features.keys())}")
print(f"  output_features: {list(cfg.policy.output_features.keys())}")

In [ ]:
# Compare: Raw batch vs Processed batch
print("=" * 80)
print("COMPARISON: Raw vs Processed Batch")
print("=" * 80)

# Get a fresh batch
raw_batch = next(dl_iter)
processed_batch = preprocessor(raw_batch.copy())

print("\n1. ACTION:")
if "action" in raw_batch:
    print(f"   Raw shape: {raw_batch['action'].shape}")
    print(f"   Raw dtype: {raw_batch['action'].dtype}")
    print(f"   Raw device: {raw_batch['action'].device}")
    print(f"   Raw value range: [{raw_batch['action'].min():.3f}, {raw_batch['action'].max():.3f}]")
if "action" in processed_batch:
    print(f"   Processed shape: {processed_batch['action'].shape}")
    print(f"   Processed dtype: {processed_batch['action'].dtype}")
    print(f"   Processed device: {processed_batch['action'].device}")
    print(f"   Processed value range: [{processed_batch['action'].min():.3f}, {processed_batch['action'].max():.3f}]")
    print("   → Actions are normalized to [-1, 1] range")

print("\n2. STATE:")
if "observation.state" in raw_batch:
    print(f"   Raw shape: {raw_batch['observation.state'].shape}")
    print(f"   Raw dtype: {raw_batch['observation.state'].dtype}")
    print(f"   Raw value range: [{raw_batch['observation.state'].min():.3f}, {raw_batch['observation.state'].max():.3f}]")
if "observation.state" in processed_batch:
    print(f"   Processed shape: {processed_batch['observation.state'].shape}")
    print(f"   Processed dtype: {processed_batch['observation.state'].dtype}")
    print(f"   Processed value range: [{processed_batch['observation.state'].min():.3f}, {processed_batch['observation.state'].max():.3f}]")
    print("   → State is normalized to [-1, 1] and discretized in prompt")

print("\n3. IMAGES:")
image_keys = [k for k in raw_batch.keys() if "image" in k.lower()]
for key in image_keys[:2]:  # Show first 2 image keys
    if key in raw_batch:
        print(f"\n   {key}:")
        print(f"   Raw shape: {raw_batch[key].shape}")
        print(f"   Raw dtype: {raw_batch[key].dtype}")
        print(f"   Raw value range: [{raw_batch[key].min():.3f}, {raw_batch[key].max():.3f}]")
    if key in processed_batch:
        print(f"   Processed shape: {processed_batch[key].shape}")
        print(f"   Processed dtype: {processed_batch[key].dtype}")
        print(f"   Processed value range: [{processed_batch[key].min():.3f}, {processed_batch[key].max():.3f}]")
        print("   → Images are resized to image_resolution and normalized to [-1, 1]")

print("\n4. LANGUAGE TOKENS:")
if "observation.language_tokens" in processed_batch:
    tokens = processed_batch["observation.language_tokens"]
    masks = processed_batch.get("observation.language_attention_mask", None)
    print(f"   Shape: {tokens.shape}")
    print(f"   Dtype: {tokens.dtype}")
    print(f"   Device: {tokens.device}")
    print(f"   Token ID range: [{tokens.min().item()}, {tokens.max().item()}]")
    if masks is not None:
        print(f"   Attention mask shape: {masks.shape}")
        print(f"   Valid tokens per sample: {masks.sum(dim=1)}")
    print("   → Prompt is tokenized using PaliGemma tokenizer")
    print("   → Format: 'Task: {subtask} State: {discretized_state};\\nAction: '")

## Understanding `cfg.steps` vs Epochs

In `train_val_pi05.py`, the training loop uses:

```python
for _ in range(step, cfg.steps):
    batch = next(dl_iter)  # Get next batch from infinite iterator
    batch = preprocessor(batch)
    # ... training step ...
    step += 1
```

**Key points:**

1. **`cfg.steps`** = Total number of training steps (gradient updates), NOT epochs
2. **`cycle(dataloader)`** creates an infinite iterator that automatically restarts when exhausted
3. This approach is common in RL/robotics because:
   - You care more about number of gradient updates than epochs
   - Datasets can be very large, so epochs might take too long
   - You can easily control training duration with a step count

**Example calculation:**
- Dataset: 10,000 samples
- Batch size: 50
- Batches per epoch: 10,000 / 50 = 200 batches
- If `cfg.steps = 1000`:
  - We do 1000 gradient updates
  - This equals 1000 / 200 = 5 epochs worth of data
  - But we don't explicitly count epochs, just steps

In [ ]:
# Obtener batch procesado (igual que next(dl_iter) en train_val_pi05.py)
batch = next(dl_iter)
batch = preprocessor(batch)